<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2022-actors/blob/czasy-joinow/colabs/Czasy_joinow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup sparka

##Setup sparka

In [1]:
!pip install pyspark py4j
!pip install -q findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget ftp://ftp.ps.pl/pub/apache/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
!tar xf spark-3.3.1-bin-hadoop2.tgz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2022-11-22 23:16:49--  ftp://ftp.ps.pl/pub/apache/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
           => ‘spark-3.3.1-bin-hadoop2.tgz.1’
Resolving ftp.ps.pl (ftp.ps.pl)... 212.14.28.36, 2001:4c58:62:2::36
Connecting to ftp.ps.pl (ftp.ps.pl)|212.14.28.36|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/apache/spark/spark-3.3.1 ... done.
==> SIZE spark-3.3.1-bin-hadoop2.tgz ... 274099817
==> PASV ... done.    ==> RETR spark-3.3.1-bin-hadoop2.tgz ... done.
Length: 274099817 (261M) (unauthoritative)

spark-3.3.1-bin-had 100%[===================>] 261.40M  22.0MB/s    in 12s     

2022-11-22 23:17:06 (21.0 MB/s) - ‘spark-3.3.1-bin-hadoop2.tgz.1’ saved [274099817]



In [2]:
import pyspark
import findspark
from pyspark.sql import SparkSession
import os

In [3]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop2"
spark=SparkSession.builder.appName('Colab').getOrCreate()
spark

#Pobranie danych

##Import danych

In [4]:
!wget https://datasets.imdbws.com/name.basics.tsv.gz
!wget https://datasets.imdbws.com/title.akas.tsv.gz
!wget https://datasets.imdbws.com/title.basics.tsv.gz
!wget https://datasets.imdbws.com/title.crew.tsv.gz
!wget https://datasets.imdbws.com/title.episode.tsv.gz
!wget https://datasets.imdbws.com/title.principals.tsv.gz
!wget https://datasets.imdbws.com/title.ratings.tsv.gz

--2022-11-22 23:17:43--  https://datasets.imdbws.com/name.basics.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 13.249.98.35, 13.249.98.61, 13.249.98.73, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|13.249.98.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 236745635 (226M) [binary/octet-stream]
Saving to: ‘name.basics.tsv.gz.1’

name.basics.tsv.gz. 100%[===================>] 225.78M  71.7MB/s    in 3.5s    

2022-11-22 23:17:47 (63.7 MB/s) - ‘name.basics.tsv.gz.1’ saved [236745635/236745635]

--2022-11-22 23:17:47--  https://datasets.imdbws.com/title.akas.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 13.249.98.35, 13.249.98.61, 13.249.98.73, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|13.249.98.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 288222028 (275M) [binary/octet-stream]
Saving to: ‘title.akas.tsv.gz.1’

title.akas.tsv.gz.1 100%[===================>] 274.87M 

##Rozpakowanie danych

In [5]:
!gzip -dc /content/name.basics.tsv.gz > name.basics.csv
!gzip -dc /content/title.akas.tsv.gz > title.akas.csv
!gzip -dc /content/title.basics.tsv.gz > title.basic.csv
!gzip -dc /content/title.crew.tsv.gz > title.crew.csv
!gzip -dc /content/title.episode.tsv.gz > title.episode.csv
!gzip -dc /content/title.principals.tsv.gz > title.principals.csv
!gzip -dc /content/title.ratings.tsv.gz > title.ratings.csv

#Wczytanie danych

##Wczytajmy dane z rozpakowanych plików

In [6]:
df_name_basics=spark.read.option("header", "true").option("delimiter", "\t").csv('name.basics.csv' ) 
df_title_akas=spark.read.option("header","true").option("delimiter", "\t").csv('title.akas.csv')
df_title_basic=spark.read.option("header","true").option("delimiter", "\t").csv('title.basic.csv')
df_title_crew=spark.read.option("header","true").option("delimiter", "\t").csv('title.crew.csv')
df_title_episode=spark.read.option("header","true").option("delimiter","\t").csv('title.episode.csv')
df_title_principals=spark.read.option("header","true").option("delimiter","\t").csv('title.principals.csv')
df_title_ratings=spark.read.option("header","true").option("delimiter","\t").csv('title.ratings.csv')

##Wyświetlmy dane

In [7]:
df_name_basics.show(3)
df_title_akas.show(3)
df_title_basic.show(3)
df_title_crew.show(3)
df_title_episode.show(3)
df_title_principals.show(3)
df_title_ratings.show(3)

+---------+---------------+---------+---------+--------------------+--------------------+
|   nconst|    primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+---------+---------------+---------+---------+--------------------+--------------------+
|nm0000001|   Fred Astaire|     1899|     1987|soundtrack,actor,...|tt0072308,tt00531...|
|nm0000002|  Lauren Bacall|     1924|     2014|  actress,soundtrack|tt0117057,tt00383...|
|nm0000003|Brigitte Bardot|     1934|       \N|actress,soundtrac...|tt0056404,tt00573...|
+---------+---------------+---------+---------+--------------------+--------------------+
only showing top 3 rows

+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
|  titleId|ordering|               title|region|language|      types|   attributes|isOriginalTitle|
+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
|tt0000001|       1|          Карменсіта|    

# Złączenie istotnych kolumn

## Wybierzmy następujące kolumny z tabel:
- df_title_basic:
  - tconst (unikalny numer tytułu - potrzebne do złączenia tabel)
  - titleType (rodzaj tytułu, np. film, serial - można porównywać podobieństwo aktorów na podstawie tego w jakich typach produkcji grali)
  - originalTitle (tytuł produkcji - być może przyda się do analizy podobnych filmów)
  - isAdult (czy produkcja jest dla dorosłych - można wykorzystać do porównywania aktorów na podstawie tego czy grają głównie w produkcjach dla dorosłych)
  - genres (gatunki - może posłużyć do porównania aktorów na podstawie tego, że grali w produkcjach o podobnych gatunkach)

- df_title_principals:
  - tconst (unikalny numer tytułu - potrzebne do złączenia tabel)
  - nconst (unikalny numer aktora - potrzebne do złączenia tabel)
  - category (kategoria pracy (actor/actress) - może posłużyć do porównania aktorów na podstawie takiej samej płci)
  - characters (postacie zagrane w produkcji - może posłużyć do porównania aktorów na podstawie podobnych granych postaci)

- df_name_basics:
  - nconst(unikalny numer aktora - potrzebne do złączenia tabel)
  - primaryName (imię i nazwisko aktora - potrzebne do wyświetlania aktora lub do odnajdywania go w bazie po imieniu i nazwisku)
  - birthYear (rok urodzenia aktora - może posłużyć do porównania aktorów na podstawie podobnych okresów, w których żyli i grali)
  - deathYear ( rok śmierci aktora - może posłużyć do porównania aktorów na podstawie podobnych okresów, w których żyli i grali)
  - knownForTitles (tytuły produkcji, z których znany jets aktor - może posłużyć do porównania aktorów na podstawie tych samych lub podobnych produkcji, z których są znani)

In [8]:
df_title_basic_selected = df_title_basic.select(["tconst", "titleType", "originalTitle", "isAdult", "genres"])
df_title_principals_selected = df_title_principals.select(["tconst", "nconst", "category", "characters"])
df_name_basics_selected = df_name_basics.select(["nconst", "primaryName", "birthYear", "deathYear", "knownForTitles"])

## Wybierzmy z tabeli df_title_principals_selected tylko aktorów i aktorki

In [9]:
#df_title_principals_selected = df_title_principals_selected.filter((df_title_principals_selected.category == "actor") | (df_title_principals_selected.category == "actress"))

# Złączmy tabele df_name_basics, df_title_principals i df_title_basic

## Sprawdźmy czasy joinowania dla różnych kombinacji tabel

Sprawdzimy najpierw jak długo zajmuje łączenie tabel bez usuniętych niepotrzebnych rekordów. Rozważmy różne kombinacje łączeń, gdzie tabele są łączone w następujący sposób:
* df_name_basics_selected i df_title_principals_selected, typ łączenia "inner",
* df_name_basics_selected i df_title_principals_selected, typ łączenia "right",
* dane wcześniej złączone i df_title_basic_selected, typ łączenia "inner",
* dane wcześniej złączone i df_title_basic_selected, typ łączenia "left",
* df_title_principals_selected i df_name_basics_selected, typ łączenia "inner",
* df_title_principals_selected i df_name_basics_selected, typ łączenia "left",
* df_title_basic_selected i df_title_principals_selected, typ łączenia "inner",
* df_title_basic_selected i df_title_principals_selected, typ łączenia "right",
* dane wcześniej złączone i df_name_basics_selected, typ łączenia "inner",
* dane wcześniej złączone i df_name_basics_selected, typ łączenia "right",
* df_title_principals_selected i df_title_basic_selected, typ łączenia "inner",
* df_title_principals_selected i df_title_basic_selected, typ łączenia "left".

In [10]:
%%time
data1 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "inner")
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data1.count(), len(data1.columns)))
data1 = data1.join(df_title_basic_selected, "tconst", "inner")
print("joined table size: ", (data1.count(), len(data1.columns)))

df_name_basics_selected and df_title_principals_selected joined table size:  (53163680, 8)
joined table size:  (53157572, 12)
CPU times: user 3.39 s, sys: 486 ms, total: 3.88 s
Wall time: 10min 54s


In [11]:
%%time
data2 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "inner")
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data2.count(), len(data2.columns)))
data2 = data2.join(df_title_basic_selected, "tconst", "left")
print("joined table size: ", (data2.count(), len(data2.columns)))

df_name_basics_selected and df_title_principals_selected joined table size:  (53163680, 8)
joined table size:  (53163680, 12)
CPU times: user 2.09 s, sys: 261 ms, total: 2.35 s
Wall time: 10min 25s


In [12]:
%%time
data3 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "right")
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data3.count(), len(data3.columns)))
data3 = data3.join(df_title_basic_selected, "tconst", "inner")
print("joined table size: ", (data3.count(), len(data3.columns)))

df_name_basics_selected and df_title_principals_selected joined table size:  (53173247, 8)
joined table size:  (53167105, 12)
CPU times: user 1.53 s, sys: 191 ms, total: 1.72 s
Wall time: 8min 43s


In [13]:
%%time
data4 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "right")
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data4.count(), len(data4.columns)))
data4 = data4.join(df_title_basic_selected, "tconst", "left")
print("joined table size: ", (data4.count(), len(data4.columns)))

df_name_basics_selected and df_title_principals_selected joined table size:  (53173247, 8)
joined table size:  (53173247, 12)
CPU times: user 1.18 s, sys: 111 ms, total: 1.29 s
Wall time: 7min 14s


In [14]:
%%time
data5 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "inner")
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data5.count(), len(data5.columns)))
data5 = data5.join(df_title_basic_selected, "tconst", "inner")
print("joined table size: ", (data5.count(), len(data5.columns)))

df_title_principals_selected and df_name_basics_selected joined table size:  (53163680, 8)
joined table size:  (53157572, 12)
CPU times: user 1.24 s, sys: 108 ms, total: 1.35 s
Wall time: 7min 18s


In [15]:
%%time
data6 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "inner")
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data6.count(), len(data6.columns)))
data6 = data6.join(df_title_basic_selected, "tconst", "left")
print("joined table size: ", (data6.count(), len(data6.columns)))

df_title_principals_selected and df_name_basics_selected joined table size:  (53163680, 8)
joined table size:  (53163680, 12)
CPU times: user 1.18 s, sys: 110 ms, total: 1.29 s
Wall time: 7min 10s


In [16]:
%%time
data7 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "left")
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data7.count(), len(data7.columns)))
data7 = data7.join(df_title_basic_selected, "tconst", "inner")
print("joined table size: ", (data7.count(), len(data7.columns)))

df_title_principals_selected and df_name_basics_selected joined table size:  (53173247, 8)
joined table size:  (53167105, 12)
CPU times: user 1.27 s, sys: 133 ms, total: 1.4 s
Wall time: 7min 20s


In [17]:
%%time
data8 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "left")
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data8.count(), len(data8.columns)))
data8 = data8.join(df_title_basic_selected, "tconst", "left")
print("joined table size: ", (data8.count(), len(data8.columns)))

df_title_principals_selected and df_name_basics_selected joined table size:  (53173247, 8)
joined table size:  (53173247, 12)
CPU times: user 1.22 s, sys: 110 ms, total: 1.33 s
Wall time: 7min


In [18]:
%%time
data9 = df_title_basic_selected.join(df_title_principals_selected, "tconst", "inner")
print("df_title_basic_selected and df_title_principals_selected joined table size: ", (data9.count(), len(data9.columns)))
data9 = data9.join(df_name_basics_selected, "nconst", "inner")
print("joined table size: ", (data9.count(), len(data9.columns)))

df_title_basic_selected and df_title_principals_selected joined table size:  (53167105, 8)
joined table size:  (53157572, 12)
CPU times: user 1.01 s, sys: 109 ms, total: 1.11 s
Wall time: 6min 9s


In [19]:
%%time
data10 = df_title_basic_selected.join(df_title_principals_selected, "tconst", "inner")
print("df_title_basic_selected and df_title_principals_selected joined table size: ", (data10.count(), len(data10.columns)))
data10 = data10.join(df_name_basics_selected, "nconst", "left")
print("joined table size: ", (data10.count(), len(data10.columns)))

df_title_basic_selected and df_title_principals_selected joined table size:  (53167105, 8)
joined table size:  (53167105, 12)
CPU times: user 1.15 s, sys: 94.6 ms, total: 1.25 s
Wall time: 6min 23s


In [20]:
%%time
data11 = df_title_basic_selected.join(df_title_principals_selected, "tconst", "right")
print("df_title_basic_selected and df_title_principals_selected joined table size: ", (data11.count(), len(data11.columns)))
data11 = data11.join(df_name_basics_selected, "nconst", "inner")
print("joined table size: ", (data11.count(), len(data11.columns)))

df_title_basic_selected and df_title_principals_selected joined table size:  (53173247, 8)
joined table size:  (53163680, 12)
CPU times: user 1.03 s, sys: 99.1 ms, total: 1.13 s
Wall time: 6min 7s


In [21]:
%%time
data12 = df_title_basic_selected.join(df_title_principals_selected, "tconst", "right")
print("df_title_basic_selected and df_title_principals_selected joined table size: ", (data12.count(), len(data12.columns)))
data12 = data12.join(df_name_basics_selected, "nconst", "left")
print("joined table size: ", (data12.count(), len(data12.columns)))

df_title_basic_selected and df_title_principals_selected joined table size:  (53173247, 8)
joined table size:  (53173247, 12)
CPU times: user 1.01 s, sys: 92.8 ms, total: 1.1 s
Wall time: 6min


In [22]:
%%time
data13 = df_title_principals_selected.join(df_title_basic_selected, "tconst", "inner")
print("df_title_principals_selected and df_title_basic_selected joined table size: ", (data13.count(), len(data13.columns)))
data13 = data13.join(df_name_basics_selected, "nconst", "inner")
print("joined table size: ", (data13.count(), len(data13.columns)))

df_title_principals_selected and df_title_basic_selected joined table size:  (53167105, 8)
joined table size:  (53157572, 12)
CPU times: user 1.08 s, sys: 98.6 ms, total: 1.17 s
Wall time: 6min 14s


In [23]:
%%time
data14 = df_title_principals_selected.join(df_title_basic_selected, "tconst", "inner")
print("df_title_principals_selected and df_title_basic_selected joined table size: ", (data14.count(), len(data14.columns)))
data14 = data14.join(df_name_basics_selected, "nconst", "left")
print("joined table size: ", (data14.count(), len(data14.columns)))

df_title_principals_selected and df_title_basic_selected joined table size:  (53167105, 8)
joined table size:  (53167105, 12)
CPU times: user 922 ms, sys: 110 ms, total: 1.03 s
Wall time: 6min 3s


In [24]:
%%time
data15 = df_title_principals_selected.join(df_title_basic_selected, "tconst", "left")
print("df_title_principals_selected and df_title_basic_selected joined table size: ", (data15.count(), len(data15.columns)))
data15 = data15.join(df_name_basics_selected, "nconst", "inner")
print("joined table size: ", (data15.count(), len(data15.columns)))

df_title_principals_selected and df_title_basic_selected joined table size:  (53173247, 8)
joined table size:  (53163680, 12)
CPU times: user 1.12 s, sys: 102 ms, total: 1.22 s
Wall time: 6min 17s


In [25]:
%%time
data16 = df_title_principals_selected.join(df_title_basic_selected, "tconst", "left")
print("df_title_principals_selected and df_title_basic_selected joined table size: ", (data16.count(), len(data16.columns)))
data16 = data16.join(df_name_basics_selected, "nconst", "left")
print("joined table size: ", (data16.count(), len(data16.columns)))

df_title_principals_selected and df_title_basic_selected joined table size:  (53173247, 8)
joined table size:  (53173247, 12)
CPU times: user 1.05 s, sys: 104 ms, total: 1.15 s
Wall time: 6min 11s


Teraz zmierzymy czasy tych samych złączeń, jednak po usunięciu z tabel niepotrzebnych rekordów.

In [26]:
df_name_basics = df_name_basics.filter("primaryProfession like '%actor%' or primaryProfession like '%actress%'")
df_name_basics_selected = df_name_basics.select(["nconst", "primaryName", "birthYear", "deathYear", "knownForTitles"])
df_title_principals_selected = df_title_principals_selected.filter((df_title_principals_selected.category == "actor") | (df_title_principals_selected.category == "actress"))

In [27]:
%%time
data1 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "inner")
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data1.count(), len(data1.columns)))
data1 = data1.join(df_title_basic_selected, "tconst", "inner")
print("joined table size: ", (data1.count(), len(data1.columns)))

df_name_basics_selected and df_title_principals_selected joined table size:  (20190998, 8)
joined table size:  (20187521, 12)
CPU times: user 805 ms, sys: 69.7 ms, total: 874 ms
Wall time: 4min 48s


In [28]:
%%time
data2 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "inner")
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data2.count(), len(data2.columns)))
data2 = data2.join(df_title_basic_selected, "tconst", "left")
print("joined table size: ", (data2.count(), len(data2.columns)))

df_name_basics_selected and df_title_principals_selected joined table size:  (20190998, 8)
joined table size:  (20190998, 12)
CPU times: user 842 ms, sys: 65.2 ms, total: 907 ms
Wall time: 4min 48s


In [29]:
%%time
data3 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "right")
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data3.count(), len(data3.columns)))
data3 = data3.join(df_title_basic_selected, "tconst", "inner")
print("joined table size: ", (data3.count(), len(data3.columns)))

df_name_basics_selected and df_title_principals_selected joined table size:  (20815083, 8)
joined table size:  (20811538, 12)
CPU times: user 790 ms, sys: 90.8 ms, total: 881 ms
Wall time: 4min 45s


In [30]:
%%time
data4 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "right")
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data4.count(), len(data4.columns)))
data4 = data4.join(df_title_basic_selected, "tconst", "left")
print("joined table size: ", (data4.count(), len(data4.columns)))

df_name_basics_selected and df_title_principals_selected joined table size:  (20815083, 8)
joined table size:  (20815083, 12)
CPU times: user 838 ms, sys: 75.8 ms, total: 914 ms
Wall time: 4min 50s


In [31]:
%%time
data5 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "inner")
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data5.count(), len(data5.columns)))
data5 = data5.join(df_title_basic_selected, "tconst", "inner")
print("joined table size: ", (data5.count(), len(data5.columns)))

df_title_principals_selected and df_name_basics_selected joined table size:  (20190998, 8)
joined table size:  (20187521, 12)
CPU times: user 781 ms, sys: 83.9 ms, total: 865 ms
Wall time: 4min 47s


In [32]:
%%time
data6 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "inner")
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data6.count(), len(data6.columns)))
data6 = data6.join(df_title_basic_selected, "tconst", "left")
print("joined table size: ", (data6.count(), len(data6.columns)))

df_title_principals_selected and df_name_basics_selected joined table size:  (20190998, 8)
joined table size:  (20190998, 12)
CPU times: user 866 ms, sys: 59.5 ms, total: 925 ms
Wall time: 4min 55s


In [33]:
%%time
data7 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "left")
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data7.count(), len(data7.columns)))
data7 = data7.join(df_title_basic_selected, "tconst", "inner")
print("joined table size: ", (data7.count(), len(data7.columns)))

df_title_principals_selected and df_name_basics_selected joined table size:  (20815083, 8)
joined table size:  (20811538, 12)
CPU times: user 797 ms, sys: 80.7 ms, total: 877 ms
Wall time: 4min 50s


In [34]:
%%time
data8 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "left")
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data8.count(), len(data8.columns)))
data8 = data8.join(df_title_basic_selected, "tconst", "left")
print("joined table size: ", (data8.count(), len(data8.columns)))

df_title_principals_selected and df_name_basics_selected joined table size:  (20815083, 8)
joined table size:  (20815083, 12)
CPU times: user 873 ms, sys: 78.3 ms, total: 951 ms
Wall time: 4min 58s


In [35]:
%%time
data9 = df_title_basic_selected.join(df_title_principals_selected, "tconst", "inner")
print("df_title_basic_selected and df_title_principals_selected joined table size: ", (data9.count(), len(data9.columns)))
data9 = data9.join(df_name_basics_selected, "nconst", "inner")
print("joined table size: ", (data9.count(), len(data9.columns)))

df_title_basic_selected and df_title_principals_selected joined table size:  (20811538, 8)
joined table size:  (20187521, 12)
CPU times: user 804 ms, sys: 69.1 ms, total: 873 ms
Wall time: 4min 40s


In [36]:
%%time
data10 = df_title_basic_selected.join(df_title_principals_selected, "tconst", "inner")
print("df_title_basic_selected and df_title_principals_selected joined table size: ", (data10.count(), len(data10.columns)))
data10 = data10.join(df_name_basics_selected, "nconst", "right")
print("joined table size: ", (data10.count(), len(data10.columns)))

df_title_basic_selected and df_title_principals_selected joined table size:  (20811538, 8)
joined table size:  (22559574, 12)
CPU times: user 772 ms, sys: 69.8 ms, total: 842 ms
Wall time: 4min 38s


In [37]:
%%time
data11 = df_title_basic_selected.join(df_title_principals_selected, "tconst", "right")
print("df_title_basic_selected and df_title_principals_selected joined table size: ", (data11.count(), len(data11.columns)))
data11 = data11.join(df_name_basics_selected, "nconst", "inner")
print("joined table size: ", (data11.count(), len(data11.columns)))

df_title_basic_selected and df_title_principals_selected joined table size:  (20815083, 8)
joined table size:  (20190998, 12)
CPU times: user 783 ms, sys: 70.2 ms, total: 854 ms
Wall time: 4min 38s


In [38]:
%%time
data12 = df_title_basic_selected.join(df_title_principals_selected, "tconst", "right")
print("df_title_basic_selected and df_title_principals_selected joined table size: ", (data12.count(), len(data12.columns)))
data12 = data12.join(df_name_basics_selected, "nconst", "right")
print("joined table size: ", (data12.count(), len(data12.columns)))

df_title_basic_selected and df_title_principals_selected joined table size:  (20815083, 8)
joined table size:  (22562970, 12)
CPU times: user 776 ms, sys: 76.1 ms, total: 852 ms
Wall time: 4min 37s


In [39]:
%%time
data13 = df_title_principals_selected.join(df_title_basic_selected, "tconst", "inner")
print("df_title_principals_selected and df_title_basic_selected joined table size: ", (data13.count(), len(data13.columns)))
data13 = data13.join(df_name_basics_selected, "nconst", "inner")
print("joined table size: ", (data13.count(), len(data13.columns)))

df_title_principals_selected and df_title_basic_selected joined table size:  (20811538, 8)
joined table size:  (20187521, 12)
CPU times: user 842 ms, sys: 113 ms, total: 955 ms
Wall time: 4min 51s


In [40]:
%%time
data14 = df_title_principals_selected.join(df_title_basic_selected, "tconst", "inner")
print("df_title_principals_selected and df_title_basic_selected joined table size: ", (data14.count(), len(data14.columns)))
data14 = data14.join(df_name_basics_selected, "nconst", "right")
print("joined table size: ", (data14.count(), len(data14.columns)))

df_title_principals_selected and df_title_basic_selected joined table size:  (20811538, 8)
joined table size:  (22559574, 12)
CPU times: user 795 ms, sys: 87.3 ms, total: 882 ms
Wall time: 4min 40s


In [41]:
%%time
data15 = df_title_principals_selected.join(df_title_basic_selected, "tconst", "left")
print("df_title_principals_selected and df_title_basic_selected joined table size: ", (data15.count(), len(data15.columns)))
data15 = data15.join(df_name_basics_selected, "nconst", "inner")
print("joined table size: ", (data15.count(), len(data15.columns)))

df_title_principals_selected and df_title_basic_selected joined table size:  (20815083, 8)
joined table size:  (20190998, 12)
CPU times: user 826 ms, sys: 71.7 ms, total: 898 ms
Wall time: 4min 39s


In [42]:
%%time
data16 = df_title_principals_selected.join(df_title_basic_selected, "tconst", "left")
print("df_title_principals_selected and df_title_basic_selected joined table size: ", (data16.count(), len(data16.columns)))
data16 = data16.join(df_name_basics_selected, "nconst", "right")
print("joined table size: ", (data16.count(), len(data16.columns)))

df_title_principals_selected and df_title_basic_selected joined table size:  (20815083, 8)
joined table size:  (22562970, 12)
CPU times: user 768 ms, sys: 67.2 ms, total: 835 ms
Wall time: 4min 33s


Sprawdzimy jakie braki mają nasze dane.

In [43]:
print(data11.filter(data11.primaryName.isNull()).count())
print(data11.filter(data11.category.isNull()).count())
print(data11.filter(data11.genres.isNull()).count())

0
0
3477


Wraźnie widać, że łączenie tabel bez zbędnych rekordów jest szybsze niż łączenie pełnych tabel. Po usunięciu zbędnych danych, różnice w czasach różnych kombinacji łączeń są już niewielkie. Ostatecznie wybieramy następujący sposób łączenia:
* df_title_basic_selected i df_title_principals_selected, typ łączenia "right"
* wcześniej złączone dane i df_name_basics_selected, typ łączenia "inner".

W danch tych nie ma żadnych nadmiarowych aktorów, pojawiają się jedynie braki co do filmów, gdyż nie wszystkie filmy, w których grali aktorzy pojawiają się w bazie.

Ten sposób łączenia danych zostanie zastosowany w notatniku głównym.


In [44]:
print("df_name_basics_selected dataframe size: ", (df_name_basics_selected.count(), len(df_name_basics_selected.columns)))
print("df_title_principals_selected dataframe size: ", (df_title_principals_selected.count(), len(df_title_principals_selected.columns)))
print("df_title_basic_selected dataframe size: ", (df_title_basic_selected.count(), len(df_title_basic_selected.columns)))
data = df_title_basic_selected.join(df_title_principals_selected, "tconst", "right")
print("joined dataframe size: ", (data.count(), len(data.columns)))
data = data.join(df_name_basics_selected, "nconst", "inner")
print("joined dataframe size: ", (data.count(), len(data.columns)))

df_name_basics_selected dataframe size:  (4428079, 5)
df_title_principals_selected dataframe size:  (20815083, 4)
df_title_basic_selected dataframe size:  (9389356, 5)
joined dataframe size:  (20815083, 8)
joined dataframe size:  (20190998, 12)


# Wybierzmy przykładowe wiersze z końcowego dataframe'u

In [45]:
sample = data.rdd.takeSample(False, 5)

In [46]:
import pandas as pd
sample = pd.DataFrame(sample, columns = data.columns)
sample

,nconst,tconst,titleType,originalTitle,isAdult,genres,category,characters,primaryName,birthYear,deathYear,knownForTitles
0,nm2670451,tt6314350,video,Rocco's Perfect Slaves 10,1,Adult,actor,\N,Mike Angelo,1981,\N,"tt1686037,tt2875866,tt5166618,tt2623476"
1,nm3065113,tt9690628,short,My Friend At The House,0,"Comedy,Short",actress,"[""Maggie""]",Laura Maynard,\N,\N,"tt1258223,tt1686907,tt9690628"
2,nm0260008,tt5929960,tvEpisode,CGI Palz: CG-Rom,0,"Animation,Comedy,Family",actor,"[""Rooty""]",John Erwin,1936,\N,"tt0089984,tt0276016,tt0112431,tt0126158"
3,nm0001147,tt0522866,tvEpisode,Misery Loves Company,0,"Drama,Romance",actress,"[""Brenda Walsh""]",Shannen Doherty,1971,\N,"tt0113749,tt0158552,tt0097493,tt0098749"
4,nm1271156,tt12021868,tvEpisode,Episode #1.531,0,Comedy,actor,"[""Various""]",Antonio Aquitania,1977,\N,"tt5807954,tt1129401,tt2995148,tt0372051"
